In [1]:
import os
import json
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.preprocessing import (MaxAbsScaler, MinMaxScaler, 
                                StandardScaler, RobustScaler)

def load_parquet_in_chunks(file_path, chunk_size=100000):
    """
    Carga un archivo parquet en chunks para manejar mejor la memoria.
    """
    parquet_file = pq.ParquetFile(file_path)
    
    # Leer el primer chunk para obtener las columnas
    first_chunk = next(parquet_file.iter_batches(batch_size=chunk_size))
    df = first_chunk.to_pandas()
    
    # Leer los chunks restantes y concatenarlos
    remaining_chunks = []
    for batch in parquet_file.iter_batches(batch_size=chunk_size):
        chunk_df = batch.to_pandas()
        remaining_chunks.append(chunk_df)
    
    if remaining_chunks:
        df = pd.concat([df] + remaining_chunks, ignore_index=True)
    
    return df

def is_boolean_column(series):
    """Detecta si una columna es booleana (solo contiene 0 y 1)."""
    unique_values = series.dropna().unique()
    return set(unique_values).issubset({0, 1})

def get_columns_to_normalize(df):
    """Identifica las columnas a normalizar: todas excepto booleanas y target."""
    columns_to_normalize = []
    
    for col in df.columns:
        if col == 'nivel_triage':  # Excluir target
            continue
        
        # Verificar si la columna es numérica
        if np.issubdtype(df[col].dtype, np.number):
            # Verificar si no es booleana
            if not is_boolean_column(df[col]):
                columns_to_normalize.append(col)
    
    return columns_to_normalize

def print_dataset_info(df, stage):
    """Imprime información sobre las columnas del dataset."""
    print(f"\n📊 {stage}")
    print("-" * 50)
    print(f"Número total de columnas: {len(df.columns)}")
    print("\nTipos de datos:")
    for dtype, count in df.dtypes.value_counts().items():
        print(f"  - {dtype}: {count} columnas")
    
    # Conteo de columnas booleanas
    boolean_cols = [col for col in df.columns if is_boolean_column(df[col])]
    print(f"\nColumnas booleanas detectadas: {len(boolean_cols)}")
    
    return {
        'total_columns': len(df.columns),
        'dtype_counts': df.dtypes.value_counts().to_dict(),
        'boolean_cols': len(boolean_cols)
    }

def validate_columns(df_original, df_normalized, method):
    """Valida que las columnas sean consistentes después de la normalización."""
    original_cols = set(df_original.columns)
    normalized_cols = set(df_normalized.columns)
    
    print(f"\n🔍 Validación de columnas para {method}:")
    print("-" * 50)
    
    if original_cols == normalized_cols:
        print("✅ Las columnas se mantienen idénticas después de la normalización")
    else:
        print("❌ Se detectaron diferencias en las columnas:")
        added = normalized_cols - original_cols
        removed = original_cols - normalized_cols
        if added:
            print(f"  Columnas agregadas: {added}")
        if removed:
            print(f"  Columnas eliminadas: {removed}")
    
    return original_cols == normalized_cols

def apply_normalization(df, columns_to_normalize, method):
    """Aplica el método de normalización especificado a las columnas seleccionadas."""
    scalers = {
        "MaxAbs": MaxAbsScaler(),
        "MinMax": MinMaxScaler(),
        "Standard": StandardScaler(),
        "Robust": RobustScaler(),
        "None": None
    }
    
    if scalers[method]:
        df[columns_to_normalize] = scalers[method].fit_transform(df[columns_to_normalize])
    return df

def save_dataframe(df, output_file, chunk_size=100000):
    """
    Guarda un DataFrame en formato parquet optimizando la memoria.
    """
    try:
        # Intentar guardar directamente primero
        df.to_parquet(output_file, index=False)
    except Exception as e:
        print(f"Advertencia: No se pudo guardar directamente, intentando con método alternativo... ({str(e)})")
        
        # Si falla, intentar con pyarrow directamente
        try:
            import pyarrow as pa
            table = pa.Table.from_pandas(df)
            pq.write_table(table, output_file)
        except Exception as e:
            print(f"Error al guardar el archivo: {str(e)}")
            raise

def main():
    try:
        # Configuración
        base_path = os.path.dirname(os.getcwd())
        config_path = os.path.join(base_path, "config.json")
            
        with open(config_path, "r") as f:
            config = json.load(f)
        input_path = os.path.join(base_path, config["paths"]["intermediate"]["featured"])
        output_path = os.path.join(base_path, config["paths"]["intermediate"]["normalized"])
        os.makedirs(output_path, exist_ok=True)
        
        # Cargar dataset con feature engineering usando chunks
        input_file = os.path.join(input_path, "df_feateng.parquet")
        print(f"\n📂 Cargando dataset desde: {input_file}")
        
        df = load_parquet_in_chunks(input_file)
        
        # Información del dataset original
        original_info = print_dataset_info(df, "Dataset Original")
        
        # Identificar columnas a normalizar
        columns_to_normalize = get_columns_to_normalize(df)
        
        print(f"\n🎯 Columnas identificadas para normalización:")
        print(f"Total columnas a normalizar: {len(columns_to_normalize)}")
        print(f"Representa el {(len(columns_to_normalize)/len(df.columns))*100:.2f}% del total de columnas")
        print("\nDesglose de columnas excluidas:")
        print(f"- Target (nivel_triage)")
        print(f"- Columnas booleanas: {len([col for col in df.columns if is_boolean_column(df[col])])}")
        
        # Diccionario para almacenar resultados
        results = {}
        
        # Aplicar todas las normalizaciones
        for method in ["MaxAbs", "MinMax", "Standard", "Robust", "None"]:
            print(f"\n🔄 Aplicando normalización: {method}")
            print("-" * 50)
            
            df_norm = df.copy()
            df_norm = apply_normalization(df_norm, columns_to_normalize, method)
            
            # Validar y guardar información
            norm_info = print_dataset_info(df_norm, f"Dataset Normalizado ({method})")
            columns_valid = validate_columns(df, df_norm, method)
            
            output_file = os.path.join(output_path, f"df_feateng_{method}.parquet")
            save_dataframe(df_norm, output_file)
            
            results[method] = {
                'output_file': output_file,
                'columns_valid': columns_valid,
                'info': norm_info
            }
            
            print(f"✅ Dataset normalizado guardado en: {output_file}")
        
        # Resumen final
        print("\n📑 Resumen Final")
        print("-" * 50)
        print(f"Dataset original: {len(df.columns)} columnas")
        print(f"Columnas normalizadas: {len(columns_to_normalize)}")
        for method, result in results.items():
            print(f"\n{method}:")
            print(f"  - Columnas totales: {result['info']['total_columns']}")
            print(f"  - Validación: {'✅' if result['columns_valid'] else '❌'}")
            print(f"  - Archivo: {os.path.basename(result['output_file'])}")
            
    except Exception as e:
        print(f"\n❌ Error durante la ejecución: {str(e)}")
        raise

if __name__ == "__main__":
    main()


📂 Cargando dataset desde: c:\Users\Administrador\Documents\PythonScripts\Tesis\tesisaustral\intermediate/featured\df_feateng.parquet

📊 Dataset Original
--------------------------------------------------
Número total de columnas: 1492

Tipos de datos:
  - float32: 740 columnas
  - int32: 608 columnas
  - float64: 144 columnas

Columnas booleanas detectadas: 383

🎯 Columnas identificadas para normalización:
Total columnas a normalizar: 1108
Representa el 74.26% del total de columnas

Desglose de columnas excluidas:
- Target (nivel_triage)
- Columnas booleanas: 383

🔄 Aplicando normalización: MaxAbs
--------------------------------------------------

❌ Error durante la ejecución: Unable to allocate 5.45 GiB for an array with shape (660486, 1108) and data type float64


MemoryError: Unable to allocate 5.45 GiB for an array with shape (660486, 1108) and data type float64